## Load in Data

In [1]:
# Import libraries
import pandas as pd
import altair as alt
import numpy as np

# import get tracking data function
from functions import get_tracking_data

# read qb_movement.csv.zip as a dataframe
tracking_df = get_tracking_data()

# Read in plays data
plays_df = pd.read_csv('./data/plays.csv')

# Read player data
players_df = pd.read_csv('./data/players.csv')

## Join Plays

In [2]:
# merge tracking data with plays to get dropbackType and yardline
tracking_df = tracking_df.merge(
    plays_df[["gameId", "playId", "dropBackType", "absoluteYardlineNumber"]],
    how="left",
    on=["gameId", "playId"],
)
# Rmeove plays that dont have a value for pass result
tracking_df = tracking_df.dropna(subset=["passResult"])
# filter for QB's
tracking_df = tracking_df[tracking_df["officialPosition"] == "QB"]
# create integer column for pass result
tracking_df["passComplete"] = (tracking_df["passResult"] == "C").astype(int)
# calculate throw distance
tracking_df["throwDistance"] = np.where(
    tracking_df["x"] < tracking_df["absoluteYardlineNumber"],
    tracking_df["rec_x"] - tracking_df["absoluteYardlineNumber"],
    np.abs(tracking_df["rec_x"] - tracking_df["absoluteYardlineNumber"]),
)

## Filter for Various Conditions

In [3]:
# filter for qb's with atleast 50 attempts and calculate there completion percentage in new column
qb_comp = tracking_df.groupby('displayName').filter(lambda x: len(x) >= 50)
qb_comp = qb_comp.groupby('displayName')['passComplete'].mean().reset_index()
qb_comp.columns = ['displayName', 'Overall']

In [4]:
# filter for traditional dropback and calculate completion percentage then merge to Qb_comp
qb_trad = tracking_df[tracking_df['dropBackType']== 'TRADITIONAL'].groupby(['displayName'])['passComplete'].mean().reset_index()
qb_trad.columns = ['displayName', 'TraditionalDropback']
qb_comp = qb_comp.merge(qb_trad, on='displayName', how='left')

In [5]:
# filter for left scrambles and rollouts and calculate completion percentage then merge to Qb_comp
tracking_left = tracking_df[(tracking_df['dropBackType']== 'SCRAMBLE_ROLLOUT_LEFT') | (tracking_df['dropBackType']== 'DESIGNED_ROLLOUT_LEFT')|((tracking_df['qb_direction']== 'left')&(tracking_df['dropBackType']== 'SCRAMBLE'))]
qb_left = tracking_left.groupby(['displayName'])['passComplete'].mean().reset_index()
qb_left.columns = ['displayName', 'GoingLeft']
qb_comp = qb_comp.merge(qb_left, on='displayName', how='left')

In [6]:
# filter for right scrambles and rollouts and calculate completion percentage then merge to Qb_comp
tracking_right = tracking_df[(tracking_df['dropBackType']== 'SCRAMBLE_ROLLOUT_RIGHT') | (tracking_df['dropBackType']== 'DESIGNED_ROLLOUT_RIGHT')|((tracking_df['qb_direction']== 'right')&(tracking_df['dropBackType']== 'SCRAMBLE'))]
qb_right = tracking_right.groupby(['displayName'])['passComplete'].mean().reset_index()
qb_right.columns = ['displayName', 'GoingRight']
qb_comp = qb_comp.merge(qb_right, on='displayName', how='left')

## Calc Avg Throw Distance

In [7]:
# calculate average throw distance for each quartback and merge to qb_comp
qb_distance = tracking_df.groupby(['displayName'])['throwDistance'].mean().reset_index()
qb_distance.columns = ['displayName', 'ReceiverDistance']
qb_comp = qb_comp.merge(qb_distance, on='displayName', how='left')

In [8]:
# Adding Attempts for each column in Qb_comp
qb_comp_count = tracking_df.groupby('displayName').filter(lambda x: len(x) >= 50)
qb_comp_count = qb_comp_count.groupby('displayName').size().reset_index(name='OverallAttempts')

qb_trad_count = tracking_df[tracking_df['dropBackType']== 'TRADITIONAL'].groupby('displayName').size().reset_index(name='TraditionalDropbackAttempts')

tracking_left_count = tracking_df[(tracking_df['dropBackType']== 'SCRAMBLE_ROLLOUT_LEFT') | (tracking_df['dropBackType']== 'DESIGNED_ROLLOUT_LEFT')|((tracking_df['qb_direction']== 'left')&(tracking_df['dropBackType']== 'SCRAMBLE'))]
qb_left_count = tracking_left_count.groupby('displayName').size().reset_index(name='GoingLeftAttempts')

tracking_right_count = tracking_df[(tracking_df['dropBackType']== 'SCRAMBLE_ROLLOUT_RIGHT') | (tracking_df['dropBackType']== 'DESIGNED_ROLLOUT_RIGHT')|((tracking_df['qb_direction']== 'right')&(tracking_df['dropBackType']== 'SCRAMBLE'))]
qb_right_count = tracking_right_count.groupby('displayName').size().reset_index(name='GoingRightAttempts')

qb_comp = qb_comp.merge(qb_comp_count, on='displayName', how='left')
qb_comp = qb_comp.merge(qb_trad_count, on='displayName', how='left')
qb_comp = qb_comp.merge(qb_left_count, on='displayName', how='left')
qb_comp = qb_comp.merge(qb_right_count, on='displayName', how='left')

## Plotting

In [9]:
# Define a function to create a single bar chart with specified color
def create_bar_chart(data, field_name, title, color, y_title,attempts, show_y_axis = True):
    chart = alt.Chart(data).mark_bar(color=color).encode(
        x=alt.X(field_name+':Q', title=title, axis=alt.Axis(orient = 'top', labels= False, ticks = False)),
        y=alt.Y('displayName:N', sort='-x', title=y_title, axis=alt.Axis(labels= show_y_axis, ticks=  show_y_axis)),
        tooltip = ['displayName', attempts]
    )
    text = chart.mark_text(
        color = 'white',
        align='right',
        baseline='middle',
        dx=-3  # Nudges text to right so it doesn't appear on top of the bar
    ).encode(
        text=alt.Text(field_name+':Q', format='.1%')
    )
    return ((chart + text).properties(height = 650, width = 150))


In [10]:
# create distance chart seperately, different text format 
chart_distance = alt.Chart(qb_comp).mark_bar(color= 'purple').encode(
        x=alt.X('ReceiverDistance:Q', title='Yards Per Attempt', axis=alt.Axis(orient = 'top', labels= False, ticks = False)),
        y=alt.Y('displayName:N', sort='-x', title=None, axis=alt.Axis(labels= False, ticks=  False)),
        tooltip = ['displayName', 'ReceiverDistance']
    )
text = chart_distance.mark_text(
    color = 'white',
    align='right',
    baseline='middle',
    dx=-3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('ReceiverDistance:Q', format='.2f')
)
chart_distance = (chart_distance + text).properties(height= 650,width = 150)

In [11]:

# Create individual charts with colors
chart_overall = create_bar_chart(qb_comp, 'Overall', 'Overall Completion %', 'blue', None, 'OverallAttempts',True )
chart_traditional = create_bar_chart(qb_comp, 'TraditionalDropback', 'Traditional Dropback Completion %', 'green', None, 'TraditionalDropbackAttempts',False)
chart_left = create_bar_chart(qb_comp, 'GoingLeft', 'Scramble/Rolling Left Completion %', 'red', None, 'GoingLeftAttempts',False)
chart_right = create_bar_chart(qb_comp, 'GoingRight', 'Scramble/Rolling Right Completion %', 'orange', None,'GoingRightAttempts', False)

# Concatenate the charts
final_chart = alt.hconcat(chart_overall, chart_traditional, chart_left, chart_right, chart_distance, spacing =0)
final_chart = final_chart.configure_axis(
    grid=False,
    labelFontWeight='bold'
).resolve_scale(y='shared').configure_view(
    stroke=None
)
final_chart = final_chart.properties(
    title={
      "text": ["Quarterback Completion Percentage Breakdown"], 
      "subtitle": ["Minimum 50 Passes thrown"],
      "subtitleColor": "gray",
      "fontSize": 20  # Adjust this value as needed
    }
)

# Display the chart
final_chart

alt.HConcatChart(...)